# Setting Up VM Instance

In [ ]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-eval-offline-{username}",
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium"
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"}
]

In [ ]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

#Retrieve code and notebooks on the instance

In [ ]:
s.execute("git clone https://github.com/shettynitis/LLM_LegalDocSummarization.git")
s.execute("cd LLM_LegalDocSummarization && git checkout Serving")

# Set up Docker

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run
```
ssh -i ~/.ssh/project_shared cc@A.B.C.D
```

# Prepare data

- On the compute instance, install rclone:

```
# run on node-persist
curl https://rclone.org/install.sh | sudo bash
```


```
# run on node-persist
# this line makes sure user_allow_other is un-commented in /etc/fuse.conf
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf
```

```
# run on node-persist
mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf # Add the file Sakshi sent
rclone lsd chi_tacc:
```

- Now present node Object Volume

```
# run on node-persist
sudo mkdir -p /mnt/LlamaData
sudo chown -R cc /mnt/LlamaData
sudo chgrp -R cc /mnt/LlamaData

# run on node-persist
rclone mount chi_tacc:object-persist-project33 /mnt/LlamaData --read-only --allow-other --daemon
  
```

# Launch a Jupyter container

```
docker run  -d --rm  -p 8888:8888 \
    -v ~/eval-offline-chi/workspace:/home/jovyan/work/ \
    -v /mnt/LlamaData:/mnt/ \
    -e LLAMA_DATA_DIR=/mnt/LlamaData \
    --name jupyter \
    quay.io/jupyter/pytorch-notebook:pytorch-2.5.1
```

```
# run on node-eval-offline
docker logs jupyter
```

